# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


**========================================================================================================================**
Import packages which will be used in this notebook:

In [1]:
import os, sys, re
import pandas as pd
import numpy as np

Load the name mapping for the column names based on SeaBird's manual:

In [2]:
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
sbe_name_map.head()

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN


**========================================================================================================================**
Declare the directory paths to where the relevant information is stored:

In [4]:
basepath = '/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-08_AR-18_2017-05-30/'
leg = 'Leg 2 (ar18b)/'
water = 'Water Sampling/'
ctd = 'ctd/'

In [5]:
bottle_path = basepath+array+cruise+leg+ctd
water_path = basepath+array+cruise+water
salts_and_o2_path = water_path+'Pioneer-08_AR-24C_2017-10-22_Oxygen_Salinity_Sample_Data/'
sample_log_path = water_path+'Pioneer-09_AR-24C_CTD_Sampling_Log.xlsx'
nutrients_path = water_path+'Pioneer-08_AR-18_Nutrients_Sample_Data_2017-08-18_ver_1-00.xlsx'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time [UTC]':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Filename':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude [degrees]':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude [degrees]':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise':cruise_id})
        else:
            pass
    
    return hdr

Get the path to the ctd-bottle data, load it, and parse it:

In [ ]:
os.listdir(bottle_path)

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(bottle_path) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(bottle_path+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
                
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
                
    #Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
    
    # Join the date and time for each measurement into a single item
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)

    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = str(cast).zfill(3)
    
    # Add the header info back in
    for key in hdr.keys():
        df[key] = hdr[key]
        
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(bottle_path+outname)
    


In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(bottle_path):
    if '.sum' in file:
        df = df.append(pd.read_csv(bottle_path+file))
    else:
        pass

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.sort_values(by=['Cast','Bottle Position'], inplace=True)
df.drop(columns='Unnamed: 0',inplace=True)
bottles = df

In [ ]:
df.to_csv(bottle_path+'CTD_Summary.csv')

**========================================================================================================================**
### Process the Discrete Salinity and Oxygen Data
Next, I process the discrete salinity and oxygen sample data so that it is consistently named and ready to be merged with the existing data sets.

In [ ]:
def clean_sal_files(dirpath):

    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for filename in os.listdir(dirpath):
            sample = []
            salinity = []
            if filename.endswith('.SAL'):
                with open(dirpath+filename) as file:
                    data = file.readlines()
                    for ind1,line in enumerate(data):
                        if ind1 == 0:
                            strs = data[0].replace('"','').split(',')
                            cruisename = strs[0]
                            station = strs[1]
                            cast = strs[2]
                            case = strs[8]
                        elif int(line.split()[0]) == 0:
                            pass
                        else:
                            strs = line.split()
                            sample.append(strs[0])
                            salinity.append(strs[2])
                
                    # Generate a pandas dataframe to populate data
                    data_dict = {'Cruise':cruisename,'Station':station,'Cast':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                    df = pd.DataFrame.from_dict(data_dict)
                    df.to_csv(file.name.replace('.','')+'.csv')
            else:
                pass
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_excel(dirpath+file))
        df.rename({'Sample':'Sample ID','Salinity':'Salinity [psu]','Niskin #':'Niskin','Case ID':'Case'}, 
                  axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for filename in os.listdir(dirpath):
        if 'oxy' in filename.lower() and filename.endswith('.xlsx'):
            df = df.append(pd.read_excel(dirpath+filename)) 
            # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Niskin #':'Niskin','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
              axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise'] = df['Cruise'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

In [ ]:
df

In [ ]:
os.listdir(salts_and_o2_path)

In [ ]:
# Now process the salts and oxygen data
    # Clean the salinity
clean_sal_files(salts_and_o2_path)
    # Process the salinity files
process_sal_files(salts_and_o2_path)
    # Process the oxygen files
process_oxy_files(salts_and_o2_path)

In [ ]:
sal = pd.read_csv(salts_and_o2_path+'SAL_Summary.csv')
sal.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
sal

In [ ]:
oxy = pd.read_csv(salts_and_o2_path+'OXY_Summary.csv')
oxy.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
oxy

**========================================================================================================================**
### CTD Sampling Log
Load in the CTD sampling log summary sheet. The summary sheet needs to be manually created and the data cleaned before attempting to import. Additionally, ensure that there is only one header line and that it is at the top of the file.

In [ ]:
os.listdir(water_path)

In [ ]:
sample_log = pd.read_excel(sample_log_path,sheet_name='Summary',header=0)
sample_log.sort_values(by=['Station-Cast #','Niskin #'])

In [ ]:
def strip_x(x):
    if type(x) == str:
        x = x.replace('.','')
        return x
    else:
        return x

In [ ]:
sample_log['Nitrate Bottle 1'] = sample_log['Nitrate Bottle 1'].apply(lambda x: strip_x(x))
sample_log['Start Date'] = sample_log['Start Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
sample_log['Start Time'] = sample_log['Start Time'].apply(lambda x: x.strftime('%H:%M:%S'))
sample_log['Start Time'] = sample_log['Start Date'] + 'T' + sample_log['Start Time'] + 'Z'

**========================================================================================================================**
### Merge the CTD-Bottle Data and Sample Log
The next step is to merge the CTD-Bottle data with the sample log using an outer merge based on the cast and niskin/bottle position. The outer merge means that all data will be retained, so that we do not accidentally discard either data-only casts or casts not recorded on the sample logs.

In [ ]:
summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #','Niskin #'], left_on=['Cast','Bottle Position'])
#summary = bottles.merge(sample_log, how='outer', right_on=['Station-Cast #'], left_on=['Cast'])

Fill in missing data based on the sample log info:

In [ ]:
summary['Start Latitude [degrees]'] = summary['Start Latitude [degrees]'].fillna(value=summary['Start Latitude'])
summary['Start Longitude [degrees]'] = summary['Start Longitude [degrees]'].fillna(value=summary['Start Longitude'])
summary['Start Time [UTC]'] = summary['Start Time [UTC]'].fillna(value=summary['Start Date']+summary['Start Time'])
summary['Station-Cast #'] = summary['Station-Cast #'].fillna(value=summary['Cast'])
summary['Bottle Position'] = summary['Bottle Position'].fillna(value=summary['Niskin #']);

Eliminate the redundant columns:

In [ ]:
summary.drop(columns=['Start Latitude','Start Longitude','Start Date','Start Time','Cast',
                      'Niskin #','Date','Time','Trip Depth'], inplace=True)

**========================================================================================================================**
Merge the discrete salinity and oxygen data into the sample_log based on the cast and niskin number. Do not use the sample bottle number - it is not stored in the processed discrete data we get back from the labs:

In [ ]:
summary = summary.merge(sal, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Station','Niskin'] )
summary['Salinity [psu]'] = summary['Salinity [psu]'].fillna(value=summary['Salts Bottle #'])
summary.rename(columns={'Salinity [psu]': 'Discrete Salinity [psu]'}, inplace=True)

Drop the unnecessary or extraneous columns:

In [ ]:
summary.drop(columns=['Cruise','Station','Niskin','Case', 'Sample ID', 'Unit', 'Salts Bottle #'], inplace=True)

Oxygen data:

In [ ]:
summary = summary.merge(oxy, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Station','Niskin'] )
summary['Oxygen [mL/L]'] =  summary['Oxygen [mL/L]'].fillna(value=summary[' Oxygen Bottle #'])
summary.rename(columns={'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'}, inplace=True)

In [ ]:
summary.drop(columns=['Cruise','Station','Niskin','Case', 'Sample ID', 'Units', ' Oxygen Bottle #'], inplace=True)

**========================================================================================================================**
### Nutrients Data
Load the nutrients data (if it exists) and merge with the summary sheet. If the nutrients data has not been returned yet, we fill in the relevant columns with the data from the sampling logs.

In [ ]:
try:
    nutrients = pd.read_excel(nutrients_path,header=0)
    nutrients
except IsADirectoryError:
    nutrients = pd.DataFrame(data=sample_log['Nitrate Bottle 1'])
    nutrients.rename(columns={'Nitrate Bottle 1':'Sample ID'}, inplace=True)
    columns = ['Sample ID','Cruise','Avg: Nitrate + Nitrite [µmol/L]','Avg: Ammonium [µmol/L]',
               'Avg: Phosphate [µmol/L]','Avg: Silicate [µmol/L]','Avg: Nitrite [µmol/L]','Avg: Nitrate [µmol/L]']
    for col in columns:
        if col not in nutrients.columns.values:
            nutrients[col] = nutrients['Sample ID']

In [ ]:
nutrients.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
summary['Nitrate Bottle 1'] = summary['Nitrate Bottle 1'].apply(lambda x: str(x).replace(' ',''))

In [ ]:
summary = summary.merge(nutrients, how='left', left_on='Nitrate Bottle 1', right_on='Sample ID')

In [ ]:
summary.info()

In [ ]:
summary.drop(columns=['Sample ID','Cruise','Nitrate Bottle 1'], inplace=True)

**========================================================================================================================**
### Chlorophyll Data
If the Chlorophyll measurements have not been returned yet, we will generate a synthetic chlorophyll spreadsheet which substitutes the sample bottle numbers in place of the actual measurements. One complication is that the Chlorophyll sample # column title is not identical between cruises.

In [ ]:
chl_path = water_path+''

In [ ]:
try:
    chl = pd.read_excel(chl_path)
    chl.head()
except IsADirectoryError:
    # If there is no chlorophyll sheet yet, need to copy the bottle data into the final sample log
    chl = sample_log[['Station-Cast #','Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube']]
    chl.rename(columns={
        'Chlorophyll Brown Bottle #': 'Brown Bottle #',
        'Chlorophyll Filter Sample #': 'Discrete Chl (ug/l)',
        'Chlorophyll LN Tube':'Discrete Phaeo (ug/l)'
    }, inplace=True)

In [ ]:
chl.dropna(subset=['Brown Bottle #'], inplace=True)

In [ ]:
summary = summary.merge(chl, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Station-Cast #','Brown Bottle #'])

In [ ]:
summary.drop(columns=['Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube','Brown Bottle #',
                     'Chlorophyll Brown Bottle Volume'], inplace = True)

**========================================================================================================================**
### Carbon-System Measurements
If the Carbon system measurements have not been returned yet, we will generate a synthetic DIC spreadsheet which substitutes the sample bottle numbers in place of the actual measurements.

In [ ]:
dic_path = water_path + ''

In [ ]:
try:
    dic = pd.read_excel(dic_path,header=0)
    dic
except IsADirectoryError:
    dic = sample_log[['Station-Cast #','Niskin #','Ph Bottle #','DIC/TA Bottle #']]
    dic.rename(columns={
        'Station-Cast #':'CAST_NO',
        'Niskin #':'NISKIN_NO',
        'DIC/TA Bottle #':'DIC_UMOL_KG',
        'Ph Bottle #':'PH_TOT_MEA',
    }, inplace=True)
    columns = ['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']
    for col in columns:
        if col not in dic.columns.values:
            if 'dic' in col.lower() or 'ta' in col.lower():
                dic[col] = dic['DIC_UMOL_KG']
            elif 'ph' in col.lower():
                dic[col] = dic['PH_TOT_MEA']
            else:
                dic[col] = np.nan

In [ ]:
dic = dic[['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']]
dic.rename(columns = {'DIC_UMOL_KG':'DIC [µmol/kg]',
               'DIC_FLAG_W':'DIC Flag',
               'TA_UMOL_KG':'Alkalinity [µmol/kg]',
               'TA_FLAG_W':'Alkalinity Flag',
               'PH_TOT_MEA':'pH [Total Scale]',
               'TMP_PH_DEG_C':'pH Analysis Temp [C]', 
              'PH_FLAG_W':'pH Flag'}, inplace=True)
# Add in the pCO2 columns, which we don't measure
dic['pCO2'] = np.nan
dic['pCO2 Flag'] = np.nan
dic['pCO2 Analysis Temp [C]'] = np.nan

dic.rename(columns=lambda x: 'Discrete ' + x, inplace=True)

In [ ]:
summary = summary.merge(dic, how='left', left_on=['Station-Cast #','Bottle Position'], right_on=['Discrete CAST_NO','Discrete NISKIN_NO'])

In [ ]:
summary.drop(columns=['Ph Bottle #','DIC/TA Bottle #','Discrete CAST_NO','Discrete NISKIN_NO'], inplace=True)

In [ ]:
summary.rename(columns={'Date Time':'Bottle Closure'}, inplace=True)

In [ ]:
summary.info();

**========================================================================================================================**
Import the column order list and use fuzzy string matching to sort the data and save the data to an new Excel spreadsheet.

In [ ]:
column_order = pd.read_excel(basepath+'column_order.xlsx')

In [ ]:
column_order = tuple([x.replace('CTD','').strip() for x in column_order.columns.values])

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 summary.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'calculated' in column.lower():
        CTDsorted[column] = -9999999
    elif 'flag' in column.lower():
        if column not in ['Discrete DIC Flag','Discrete Alkalinity Flag','Discrete pCO2 Flag','Discrete pH Flag']:
            CTDsorted[column] = -9999999
        else:
            CTDsorted[column] = summary[column]
            results.update({column:match[0]})
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif (match[0][0] not in [x[0] for x in results.values()]):
        CTDsorted[match[0][0]] = summary[match[0][0]]
        results.update({column:match[0]})
    elif len(match) == 1:
        CTDsorted[match[0][0]] = summary[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = summary[match[1][0]]
        results.update({column:match[1]})
CTDsorted['Comments'] = summary['Comments']

In [ ]:
cruise_id = list(set(CTDsorted['Cruise ID'].dropna()))
CTDsorted['Cruise ID'] = CTDsorted['Cruise ID'].fillna(value=cruise_id[0])

In [ ]:
cruise_name = cruise.replace('/','')
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-01'

In [ ]:
cruise_id

In [ ]:
filename = '_'.join([cruise_name,cruise_id[0],'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version,'.xlsx'])
filename

In [ ]:
CTDsorted.drop_duplicates(inplace=True)

In [ ]:
CTDsorted

In [ ]:
CTDsorted.fillna(value=-9999999,inplace=True)

In [ ]:
CTDsorted.to_excel(basepath+array+cruise+filename)

**========================================================================================================================**


In [6]:
summary_name = 'Pioneer-08_AR-18_Discrete_Summary_2019-06-25_ver_1-01_.xlsx'

In [7]:
summary = pd.read_excel(basepath+array+cruise+summary_name)

In [13]:
cols = [x for x in summary.columns.values if 'unnamed' in x.lower()]

In [15]:
summary.drop(columns=cols, inplace=True)

In [16]:
summary.head(10)

,Cruise ID,Station-Cast #,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth [m],CTD Filename,...,Calculated Alkalinity [µmol/kg],Calculated DIC [µmol/kg],Calculated pCO2 [µatm],Calculated pH,Calculated CO2aq [µmol/kg],Calculated bicarb [µmol/kg],Calculated CO3 [µmol/kg],Calculated Omega-C,Calculated Omega-A,Comments
0,AR18-A,1,-9999999,40 14.96 N,070 45.01 W,2017-05-31T07:56:13Z,1,100,-9999999,D:\Data\ar18a001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
1,AR18-A,2,-9999999,40 21.97 N,070 53.30 W,2017-05-31T09:30:51Z,2,100,-9999999,D:\Data\ar18a002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
2,AR18-A,3,-9999999,40 22.53 N,070 46.01 W,2017-06-01T03:36:26Z,3,100,-9999999,D:\Data\ar18a003.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
3,AR18-A,4,-9999999,40 22.49 N,070 49.99 W,2017-06-01T04:32:39Z,4,100,-9999999,D:\Data\ar18a004.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
4,AR18-A,5,-9999999,40 17.01 N,070 50.02 W,2017-06-01T05:40:35Z,5,100,-9999999,D:\Data\ar18a005.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
5,AR18-A,6,-9999999,40 12.01 N,070 50.01 W,2017-06-01T06:44:51Z,6,100,-9999999,D:\Data\ar18a006.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
6,AR18-A,7,-9999999,40 06.07 N,070 50.01 W,2017-06-01T08:01:31Z,7,100,-9999999,D:\Data\ar18a007.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
7,AR18-A,8,-9999999,40 06.27 N,070 53.65 W,2017-06-01T08:53:41Z,8,100,-9999999,D:\Data\ar18a008.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
8,AR18-A,9,OSPM,39 55.80 N,070 53.40 W,2017-06-01T20:41:30Z,9,100,469,D:\Data\ar18a009.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,NaN
9,AR18-A,9,OSPM,39 55.80 N,070 53.40 W,2017-06-01T20:41:30Z,9,100,469,D:\Data\ar18a009.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,Duplicate DIC/TA


In [18]:
summary['Discrete Nitrates [µmol/L]']

0      -9999999
1      -9999999
2      -9999999
3      -9999999
4      -9999999
5      -9999999
6      -9999999
7      -9999999
8           9-1
9           9-2
10          9-3
11     -9999999
12          9-4
13          9-5
14          9-6
15     -9999999
16     -9999999
17     -9999999
18     -9999999
19     -9999999
20     -9999999
21     -9999999
22     -9999999
23     -9999999
24     -9999999
25     -9999999
26     -9999999
27     -9999999
28          2-1
29          2-2
         ...   
147    -9999999
148         4-1
149    -9999999
150         4-2
151         4-3
152         4-4
153    -9999999
154         4-5
155         4-6
156    -9999999
157    -9999999
158    -9999999
159    -9999999
160    -9999999
161    -9999999
162    -9999999
163    -9999999
164    -9999999
165    -9999999
166    -9999999
167    -9999999
168    -9999999
169    -9999999
170    -9999999
171    -9999999
172    -9999999
173    -9999999
174    -9999999
175    -9999999
176    -9999999
Name: Discrete Nitrite [

In [19]:
# IMport the nutrients data

In [22]:
os.listdir(basepath+array+cruise+water)

['Pioneer-08_AR-18A_Oxygen_Salinity_Sample_Data',
 'Pioneer-08_AR-18B_CTD_Sampling_Log.xlsx',
 'Pioneer-08_AR-18C_CTD_sampling_log.xlsx',
 'Pioneer-08_AR-18A_CTD_Sampling_Log.xlsx',
 'Pioneer-08_AR-18_Nutrients_Sample_Data_2017-08-18_ver_1-00.xlsx',
 'Pioneer-08_AR-18B_Oxygen_Salinity_Sample_Data',
 'Pioneer-08_AR-18C_Oxygen_Salinity_Sample_Data']

In [23]:
nutrients_path = water_path+'Pioneer-08_AR-18_Nutrients_Sample_Data_2017-08-18_ver_1-00.xlsx'

In [24]:
nutrients = pd.read_excel(nutrients_path)

In [25]:
nutrients

,Sample ID,Cruise,Avg: Nitrate + Nitrite [µmol/L],Avg: Ammonium [µmol/L],Avg: Phosphate [µmol/L],Avg: Silicate [µmol/L],Avg: Nitrite [µmol/L],Avg: Nitrate [µmol/L]
0,6-1,AR18-B,18.4056,0.6660,1.09619,11.3763,<0.04,18.3656
1,6-2,AR18-B,13.6015,0.8160,0.87714,8.70838,<0.04,13.5615
2,6-3,AR18-B,14.0576,1.0020,0.968409,7.49593,<0.04,14.0176
3,6-4,AR18-B,<0.04,1.6800,<0.009,<0.03,<0.04,<0.04
4,6-5,AR18-B,<0.04,1.1940,0.0134501,<0.03,<0.04,<0.04
5,6-6,AR18-B,<0.04,1.9900,<0.009,<0.03,<0.04,<0.04
6,7-1,AR18-B,20.2275,1.4500,1.15863,11.0833,<0.04,20.1875
7,7-2,AR18-B,17.8869,1.6170,1.05199,9.79746,<0.04,17.8469
8,7-3,AR18-B,10.5906,0.4570,0.582679,4.20051,<0.04,10.5506
9,7-4,AR18-B,<0.04,1.2330,0.0317039,<0.03,<0.04,<0.04


In [27]:
summary = summary.merge(nutrients, how='left', left_on='Discrete Nitrate [µmol/L]', right_on='Sample ID')

In [29]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 212
Data columns (total 85 columns):
Cruise ID                                        213 non-null object
Station-Cast #                                   213 non-null int64
Target Asset                                     213 non-null object
Start Latitude [degrees]                         213 non-null object
Start Longitude [degrees]                        213 non-null object
Start Time [UTC]                                 213 non-null object
Cast                                             213 non-null int64
Cast Flag                                        213 non-null int64
Bottom Depth [m]                                 213 non-null int64
CTD Filename                                     213 non-null object
CTD File Flag                                    213 non-null int64
Bottle Position                                  213 non-null int64
Niskin Flag                                      213 non-null int64
CTD B

In [30]:
# Rename the columns:
summary['Discrete Nitrate [µmol/L]'] = summary['Avg: Nitrate [µmol/L]']
summary['Discrete Nitrite [µmol/L]'] = summary['Avg: Nitrite [µmol/L]']
summary['Discrete Phosphate [µmol/L]'] = summary['Avg: Phosphate [µmol/L]']
summary['Discrete Ammonium [µmol/L]'] = summary['Avg: Ammonium [µmol/L]']
summary['Discrete Silicate [µmol/L]'] = summary['Avg: Silicate [µmol/L]']

In [32]:
nutrients.columns.values

array(['Sample ID', 'Cruise', 'Avg: Nitrate + Nitrite [µmol/L]',
       'Avg: Ammonium [µmol/L]', 'Avg: Phosphate [µmol/L]',
       'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]',
       'Avg: Nitrate [µmol/L]'], dtype=object)

In [33]:
summary.drop(columns=nutrients.columns.values, inplace=True)

In [34]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 212
Data columns (total 77 columns):
Cruise ID                                        213 non-null object
Station-Cast #                                   213 non-null int64
Target Asset                                     213 non-null object
Start Latitude [degrees]                         213 non-null object
Start Longitude [degrees]                        213 non-null object
Start Time [UTC]                                 213 non-null object
Cast                                             213 non-null int64
Cast Flag                                        213 non-null int64
Bottom Depth [m]                                 213 non-null int64
CTD Filename                                     213 non-null object
CTD File Flag                                    213 non-null int64
Bottle Position                                  213 non-null int64
Niskin Flag                                      213 non-null int64
CTD B

In [35]:
summary.drop_duplicates(inplace=True)

In [36]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 212
Data columns (total 77 columns):
Cruise ID                                        213 non-null object
Station-Cast #                                   213 non-null int64
Target Asset                                     213 non-null object
Start Latitude [degrees]                         213 non-null object
Start Longitude [degrees]                        213 non-null object
Start Time [UTC]                                 213 non-null object
Cast                                             213 non-null int64
Cast Flag                                        213 non-null int64
Bottom Depth [m]                                 213 non-null int64
CTD Filename                                     213 non-null object
CTD File Flag                                    213 non-null int64
Bottle Position                                  213 non-null int64
Niskin Flag                                      213 non-null int64
CTD B

In [37]:
cols = [x for x in summary.columns.values if 'flag' in x.lower()]
cols

['Cast Flag',
 'CTD File Flag',
 'Niskin Flag',
 'CTD Pressure Flag',
 'CTD Temperature 1 Flag',
 'CTD Temperature 2 Flag',
 'CTD Conductivity 1 Flag',
 'CTD Conductivity 2 Flag',
 'CTD Oxygen Flag',
 'CTD Fluorescence Flag',
 'CTD Transmissometer Flag',
 'CTD pH Flag',
 'Discrete Oxygen Flag',
 'Discrete Oxygen Duplicate Flag',
 'Discrete Fluorescence Flag',
 'Discrete Fluorescence Duplicate Flag',
 'Discrete Nutrients Flag',
 'Discrete Nutrients Duplicate Flag',
 'Discrete Salinity Flag',
 'Discrete Salinity Duplicate Flag',
 'Discrete Alkalinity Flag',
 'Discrete DIC Flag',
 'Discrete pCO2 Flag',
 'Discrete pH Flag']

In [45]:
summary.fillna(value=-9999999, inplace=True)

In [69]:
def fill_flags(x):
    
    if x==-9999999:
        return x
    else:
        x = str(x).zfill(16)
        return x

In [70]:
for c in cols:
    print(c)
    summary[c] = summary[c].apply(lambda x: fill_flags(x))

Cast Flag
CTD File Flag
Niskin Flag
CTD Pressure Flag
CTD Temperature 1 Flag
CTD Temperature 2 Flag
CTD Conductivity 1 Flag
CTD Conductivity 2 Flag
CTD Oxygen Flag
CTD Fluorescence Flag
CTD Transmissometer Flag
CTD pH Flag
Discrete Oxygen Flag
Discrete Oxygen Duplicate Flag
Discrete Fluorescence Flag
Discrete Fluorescence Duplicate Flag
Discrete Nutrients Flag
Discrete Nutrients Duplicate Flag
Discrete Salinity Flag
Discrete Salinity Duplicate Flag
Discrete Alkalinity Flag
Discrete DIC Flag
Discrete pCO2 Flag
Discrete pH Flag


In [72]:
summary

,Cruise ID,Station-Cast #,Target Asset,Start Latitude [degrees],Start Longitude [degrees],Start Time [UTC],Cast,Cast Flag,Bottom Depth [m],CTD Filename,...,Calculated Alkalinity [µmol/kg],Calculated DIC [µmol/kg],Calculated pCO2 [µatm],Calculated pH,Calculated CO2aq [µmol/kg],Calculated bicarb [µmol/kg],Calculated CO3 [µmol/kg],Calculated Omega-C,Calculated Omega-A,Comments
0,AR18-A,1,-9999999,40 14.96 N,070 45.01 W,2017-05-31T07:56:13Z,1,0000000000000100,-9999999,D:\Data\ar18a001.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
1,AR18-A,2,-9999999,40 21.97 N,070 53.30 W,2017-05-31T09:30:51Z,2,0000000000000100,-9999999,D:\Data\ar18a002.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
2,AR18-A,3,-9999999,40 22.53 N,070 46.01 W,2017-06-01T03:36:26Z,3,0000000000000100,-9999999,D:\Data\ar18a003.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
3,AR18-A,4,-9999999,40 22.49 N,070 49.99 W,2017-06-01T04:32:39Z,4,0000000000000100,-9999999,D:\Data\ar18a004.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
4,AR18-A,5,-9999999,40 17.01 N,070 50.02 W,2017-06-01T05:40:35Z,5,0000000000000100,-9999999,D:\Data\ar18a005.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
5,AR18-A,6,-9999999,40 12.01 N,070 50.01 W,2017-06-01T06:44:51Z,6,0000000000000100,-9999999,D:\Data\ar18a006.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
6,AR18-A,7,-9999999,40 06.07 N,070 50.01 W,2017-06-01T08:01:31Z,7,0000000000000100,-9999999,D:\Data\ar18a007.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
7,AR18-A,8,-9999999,40 06.27 N,070 53.65 W,2017-06-01T08:53:41Z,8,0000000000000100,-9999999,D:\Data\ar18a008.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
8,AR18-A,9,OSPM,39 55.80 N,070 53.40 W,2017-06-01T20:41:30Z,9,0000000000000100,469,D:\Data\ar18a009.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999
9,AR18-A,9,OSPM,39 55.80 N,070 53.40 W,2017-06-01T20:41:30Z,9,0000000000000100,469,D:\Data\ar18a009.hex,...,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999,-9999999


In [65]:
cruise_name = cruise.split('_')[0]
cruise_name

'Pioneer-08'

In [51]:
summary['Cruise ID'] = summary['Cruise ID'].fillna(value=cruise_id[0])

In [57]:
cruise
cruise_id = list(set(summary['Cruise ID'].dropna()))[0].split('-')[0]
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-02'

In [66]:
filename = '_'.join([cruise_name,cruise_id,'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version])
filename = filename+'.csv'
filename

'Pioneer-08_AR18_Discrete_Summary_2019-06-27_ver_1-02.csv'

In [68]:
summary.to_csv(basepath+array+cruise+filename)